## 4. Political blogs dataset

In [1]:
import csv
from collections import defaultdict
from statistics import mode, mean
import numpy as np

In [3]:
def run_spectral_clustering(L, k):
    v, x = np.linalg.eig(L)
    idx = np.argsort(v)
    x = np.real(x[:, idx[-k:]])
    x = x / np.repeat(np.sqrt(np.sum(x * x, axis=1).reshape(-1, 1)), k, axis=1)
    centroids = x[np.random.choice(x.shape[0], k, replace=False)]
    cluster_labels = np.zeros(x.shape[0], dtype=int)

    for _ in range(100):
        distances = np.linalg.norm(x[:, np.newaxis] - centroids, axis=2)
        new_labels = np.argmin(distances, axis=1)
        new_centroids = np.array([x[new_labels == i].mean(axis=0) for i in range(k)])
        if np.all(cluster_labels == new_labels):
            break
        centroids = new_centroids
        cluster_labels = new_labels

    return cluster_labels

def calc_metrics(nodes, labels):
    num_nodes = len(nodes)
    cluster_groups = defaultdict(list)
    majority = {}
    mismatch = {}

    for i in range(num_nodes):
        cluster = labels[i]
        group = nodes[i][2]
        cluster_groups[cluster].append(group)

    for key in cluster_groups.keys():
        majority[key] = mode(cluster_groups[key])
        value = cluster_groups[key]
        mismatch[key] = round(1 - value.count(majority[key]) / len(value), 4)

    return majority, mismatch

In [6]:
with open("q4_data/nodes.txt") as f:
    reader = csv.reader(f, delimiter="\t")
    nodes0 = []
    for row in reader:
        nodes0.append([int(row[0]), row[1], int(row[2])])

with open("q4_data/edges.txt") as f:
    reader = csv.reader(f, delimiter="\t")
    edges = np.array([list(map(int, row)) for row in reader])

n = len(nodes0)
A = np.zeros((n, n), dtype=int)

for edge in edges:
    A[edge[0] - 1, edge[1] - 1] = 1
    A[edge[1] - 1, edge[0] - 1] = 1

d_i = A.sum(axis=0)
D = np.diag(d_i)

del_list = [i for i in range(len(d_i)) if d_i[i] == 0]
A = np.delete(A, del_list, axis=0)
A = np.delete(A, del_list, axis=1)
D = np.delete(D, del_list, axis=0)
D = np.delete(D, del_list, axis=1)
nodes = [nodes0[i] for i in range(len(nodes0)) if i not in del_list]

L = D - A

### Q1

In [7]:
k_vals = [2, 5, 10, 30, 50]

for k in k_vals:
    labels = run_spectral_clustering(L, k)
    majority, mismatch = calc_metrics(nodes, labels)

    avg = mean(mismatch.values())
    min_rate = min(mismatch.values())
    max_rate = max(mismatch.values())

    print("k = " + str(k))
    print("Majority: " + str(dict(sorted(majority.items()))))
    print("Mismatch Rates: " + str(dict(sorted(mismatch.items()))))
    print("Average of Mismatch Rates = " + str(avg))
    print(
        "Spread of Mismatch Rates = ("
        + str(min_rate)
        + ", "
        + str(max_rate)
        + ") = "
        + str(max_rate - min_rate)
    )
    print("")

k = 2
Majority: {0: 1, 1: 0}
Mismatch Rates: {0: 0.2919, 1: 0.3366}
Average of Mismatch Rates = 0.31425000000000003
Spread of Mismatch Rates = (0.2919, 0.3366) = 0.04470000000000002

k = 5
Majority: {0: 1, 1: 1, 2: 0, 3: 0, 4: 0}
Mismatch Rates: {0: 0.0173, 1: 0.0769, 2: 0.4922, 3: 0.0753, 4: 0.15}
Average of Mismatch Rates = 0.16234
Spread of Mismatch Rates = (0.0173, 0.4922) = 0.47490000000000004

k = 10
Majority: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0}
Mismatch Rates: {0: 0.0331, 1: 0.2632, 2: 0.1368, 3: 0.13, 4: 0.363, 5: 0.1053, 6: 0.0683, 7: 0.0676, 8: 0.0275, 9: 0.0471}
Average of Mismatch Rates = 0.12419
Spread of Mismatch Rates = (0.0275, 0.363) = 0.33549999999999996

k = 30
Majority: {0: 1, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 0, 13: 1, 14: 0, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 1}
Mismatch Rates: {0: 0.2, 1: 0.0, 2: 0.0351, 3: 0.1429, 4: 0.14, 5: 0.0, 6: 

### Q2

In [ ]:
best_k = None
best_mismatch_rates = None
lowest_avg_mismatch = float("inf")

for k in range(2, 10):
    labels = run_spectral_clustering(L, k)
    majority, mismatch = calc_metrics(nodes, labels)

    avg = mean(mismatch.values())
    min_rate = min(mismatch.values())
    max_rate = max(mismatch.values())

    print("k = " + str(k))
    print("Majority: " + str(dict(sorted(majority.items()))))
    print("Mismatch Rates: " + str(dict(sorted(mismatch.items()))))
    print("Average of Mismatch Rates = " + str(avg))
    print("Spread of Mismatch Rates = (" + str(min_rate) + ", " + str(max_rate) + ") = " + str(max_rate - min_rate))
    print("")

    if avg < lowest_avg_mismatch:
        best_k = k
        best_mismatch_rates = mismatch
        lowest_avg_mismatch = avg

print(f"Mismatch rates for {best_k}:")
for cluster, rate in sorted(best_mismatch_rates.items()):
    print(f"Cluster {cluster} = {rate}")

k = 2
Majority: {0: 1, 1: 0}
Mismatch Rates: {0: 0.449, 1: 0.4636}
Average of Mismatch Rates = 0.45630000000000004
Spread of Mismatch Rates = (0.449, 0.4636) = 0.014600000000000002

k = 3
Majority: {0: 1, 1: 0, 2: 0}
Mismatch Rates: {0: 0.0624, 1: 0.45, 2: 0.0912}
Average of Mismatch Rates = 0.2012
Spread of Mismatch Rates = (0.0624, 0.45) = 0.3876

k = 4
Majority: {0: 1, 1: 1, 2: 0, 3: 0}
Mismatch Rates: {0: 0.0167, 1: 0.1379, 2: 0.4769, 3: 0.0543}
Average of Mismatch Rates = 0.17145
Spread of Mismatch Rates = (0.0167, 0.4769) = 0.4602

k = 5
Majority: {0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
Mismatch Rates: {0: 0.0904, 1: 0.0175, 2: 0.075, 3: 0.1311, 4: 0.0656}
Average of Mismatch Rates = 0.07592
Spread of Mismatch Rates = (0.0175, 0.1311) = 0.11359999999999999

k = 6
Majority: {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0}
Mismatch Rates: {0: 0.3026, 1: 0.3379, 2: 0.1062, 3: 0.2747, 4: 0.0474, 5: 0.0558}
Average of Mismatch Rates = 0.18743333333333334
Spread of Mismatch Rates = (0.0474, 0.3379) = 0.290